<a href="https://colab.research.google.com/github/emmiduh/ai_agents_mcp_examples/blob/main/ai_agents_mcp_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

agent.py

In [7]:
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata


API_KEY = userdata.get('GEMINI_API_KEY')

if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please add it in Colab Secrets.")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

print("Welcome to your AI Assistant. Type 'goodbye' to quit.\n")

while True:
    prompt = input("You: ")
    if prompt.lower() == "goodbye":
        print("\nAI Assistant: Goodbye!")
        break
    response = model.generate_content(prompt)

    print("AI Assistant:", response.text)

Welcome to your AI Assistant. Type 'goodbye' to quit.

You: goodbye

AI Assistant: Goodbye!


client.py for stdio transport

In [8]:
from typing import Any
from contextlib import AsyncExitStack
from mcp import ClientSession
from mcp.client.stdio import StdioServerParameters, stdio_client

class MCPClient:
    """MCP Client class for connecting to and interacting with MCP servers."""

    def __init__(self,
                 name: str,
                 command: str,
                 server_args: list[str],
                 env_vars: dict[str, str] = None,
                 ) -> None:
        """Initialize the MCPClient."""
        self.name = name
        self.command = command
        self.server_args = server_args
        self.env_vars = env_vars
        self._session: ClientSession = None
        self._exit_stack: AsyncExitStack = AsyncExitStack()
        self._connected: bool = False

    async def connect(self) -> None:
        """Connect to the server set in the constructor."""
        if self.connected:
          raise RuntimeError("Client is already connected")

        server_parameters = StdioServerParameters(
            command=self.command,
            args=self.server_args,
            env=self.env_vars if self.env_vars else None
        )

        # Connect to stdio server, starting subprocess
        stdio_connection = await self._exit_stack.enter_async_context(
            stdio_client(server_parameters)
        )
        self.read, self.write = stdio_connection

        # Start MCP client session
        self.session = await self._exit_stack.enter_async_context(
            ClientSession(read_stream=self.read, write_stream=self.write)
        )

        # Initialize session
        await self._session.initialize()
        self._connected = True

    async def get_available_tools(self) -> list[Any]:
        """Retrieve tools that the server has made available."""
        pass

    async def use_tool(self, tool_name: str, tool_args: list | None = None):
        """Given a tool name and optionally a list of argumnents, execute the tool."""
        pass

    async def disconnect(self) -> None:
        """Clean up any resources."""
        if self._exit_stack:
          await self._exit_stack.aclose()
          self._connected = False
          self._session = None

client.py for streamable http transport

In [9]:
from contextlib import AsyncExitStack
from typing import Callable
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

class MCPClient:
    def __init__(self, name: str, server_url: str) -> None:
        self.name = name
        self.server_url = server_url
        self._session: ClientSession = None
        self._exit_stack = AsyncExitStack()
        self._connected: bool = False
        self._get_session_id: Callable[[], str] = None

    async def connect(self, headers: dict | None = None) -> None:
        if self.connected:
          raise RuntimeError("Client is already connected")

        # Connect to Streamable HTTP server
        streamable_connection = await self._exit_stack.enter_async_context(streamablehttp_client(url=self.server_url, headers=headers))
        self.read, self.write, self._get_session_id = streamable_connection

        # Start MCP client session
        self._session = await self._exit_stack.enter_async_context(ClientSession(read_stream=self.read, write_stream=self.write))

        # Intialize session
        await self._session.initialize()
        self._connected = True

    async def disconnect(self) -> None:
        """Clean up any resources."""
        if self._exit_stack:
          await self._exit_stack.aclose()
          self._connected = False
          self._session = None


instantiate stdio client: updated agent.py

In [10]:
import asyncio
from pathlib import Path

import google.generativeai as genai
from client import MCPClient


API_KEY = userdata.get('GEMINI_API_KEY')

if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please add it in Colab Secrets.")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

mcp_client = MCPClient(
    name="calculator_server_connection",
    command="uv",
    server_args=[
        "--directory",
        str(Path(__file__).parent.parent.resolve()),
        "run",
        "calculator_server.py",
    ],
)

print("Welcome to your AI Assistant. Type 'goodbye' to quit.\n")

async def main():
    """Main async function to run the assistant."""
    await mcp_client.connect()
    available_tools = await mcp_client.get_available_tools()
    print(
        f"Available tools: {', '.join([tool['name'] for tool in available_tools])}"
    )

    while True:
        prompt = input("You: ")
        if prompt.lower() == "goodbye":
            print("AI Assistant: Goodbye!")
            break

        response = model.generate_content(prompt)
        print(f"Assistant: {response.text}")

    await mcp_client.disconnect()


if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'client'